#
<b>

==============================

Nama: Muhammad Iqbal Saputra

Batch: RMT-032

Objektif dari program ini adalah:
- Mencari mean, median dan modus dari dataset
- Mencari nilai skew
- Menghilangkan outliers/extreme value analysis
- Menyimpan data yang sudah dihilangkan outliersnya

==============================

In [1]:
# Proses autentikasi
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "ace-mile-424512-u5"
client = bigquery.Client(project=project_id)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Hanya mengambil kolom 'price' yang mempunyai value dari kolom operation 'rent' dan property_type 'apartment'
df = client.query('''
SELECT price
FROM `bigquery-public-data.properati_properties_ar.properties_rent_201501`
WHERE operation = 'rent' AND property_type = 'apartment'
LIMIT 1500
''').to_dataframe()

df.head()

# Analisis Anomali

## 1. Mean, Median, Modus

In [ ]:
# Mencari nilai rata-rata, median dan modus dari dataset
mean = df['price'].mean()
median = df['price'].median()
mode = df['price'].mode().values[0]

print(f"Mean = {mean}")
print(f"Median = {median}")
print(f"Mode = {mode}")

## 2. Skewness

In [ ]:
# Mencari nilai skew dari dataset
skewness = df['price'].skew()

print(f"Skew = {skewness}")

hasil skew positif yang berarti data cenderung tertarik kekanan karna frekuensi value 'price' yang lebih tinggi ternyata lebih banyak

## 3. Extreme Value Analysis

In [ ]:
# Menghilangkan outliers pada dataset
q1 = df['price'].quantile(0.25)
q2 = df['price'].quantile(0.5)
q3 = df['price'].quantile(0.75)

iqr = q3 - q1

lower_bound = q1 - 1.5*iqr
upper_bound = q3 + 1.5*iqr

outlier = df['price'][(df['price'] < lower_bound) | (df['price'] > upper_bound)]
no_outlier = df['price'][(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

print(f'Percentage of outliers: {len(outlier)*100/len(df["price"]):.2f}%')



outliers masih banyak

## 4. Membuat variabel baru utk menyimpan data yang sudah dibuang anomalinya

In [ ]:
# Melakukan iterasi untuk menghilangkan outliers sampai batas maksimum outliers
max_outliers_percent = 5
percent_outliers = 100
df_clean = df.copy() # variabel baru utk menyimpan data yang sudah bersih

while percent_outliers > max_outliers_percent:
  q1 = df_clean['price'].quantile(0.25)
  q3 = df_clean['price'].quantile(0.75)
  iqr = q3-q1

  lower_bound = q1 - 1.5*iqr
  upper_bound = q3 + 1.5*iqr

  outliers = df_clean[(df_clean['price']< lower_bound) | (df_clean['price'] > upper_bound)]
  percent_outliers = len(outliers) * 100 / len(df_clean['price'])

  if percent_outliers > max_outliers_percent:
    df_clean = df_clean[(df_clean['price'] >= lower_bound) & (df_clean['price'] <= upper_bound)]


print(f'Final percentage of outliers: {percent_outliers:.0f}%')

In [ ]:
df_clean.info()

Data tersortir dari 1500 menjadi 781 setelah dihilangkan outliersnya

In [ ]:
skew = df_clean['price'].skew()

print(f"Skew df_clean = {skew}")

Nilai skew sudah tidak se-ekstrem sebelumnya

## 5. Menyimpan data yang sudah bersih

In [ ]:
# Mengimpor data yang sudah dibersihkan
df.to_csv('h8dsft_P0LC3_iqbal_saputra.csv', index=False)
from google.colab import files
files.download('h8dsft_P0LC3_iqbal_saputra.csv')

# Pertanyaan

1. Sebelum dihilangkan outliersnya data memiliki:
- Mean = 6175.914141414141
- Median = 3200.0
- Mode = 0.0

Menurut analisis saya, kecenderungan data lebih memusat kekanan dikarenakan frekuensi value price yang lebih tinggi(nilai ekstrim) lebih banyak dibandingkan value price yang lebih kecil(nilai ekstrim), yang berarti umumnya harga sewa apartement lebih tinggi dari harga median dan rata-rata


2. Sebelum melakukan extreme value analysis, nilai skewness pada data adalah 3.99 yang berarti distribusi data lebih cenderung ke value(price) yang lebih tinggi

* -0.5 < skew < 0.5 = simetris
* -1 < skew < 1 = slightly skew
* skew < -1 | 1 < skew = moderate skew

3. Dikarenakan nilai skew pada data > ±0.5, maka saya menggunakan metode Tukey's rule atau IQR, karna metode tersebut lebih robust untuk data yang terdapat banyak outliers